In [16]:
import geopandas as gpd
from shapely.geometry import mapping
import zipfile
import tempfile
from pathlib import Path
from shapely.validation import make_valid

# cell 0
# GitHub Copilot
# Compare area of GeoJSON (#1) covered by shapefile set in a ZIP (#3).
# Assumes these files are in the notebook working directory:
# - "wsp_osm_city_pc_boundaries_not_in_b1.geojson"
# - "EagleView_WA Flight Coverage.zip"
#
# Output: prints total area (m^2 and km^2), covered area, uncovered area,
# and writes two GeoJSONs: covered.geojson and not_covered.geojson.

import shapely.ops as ops

# Input file paths (adjust if different)
# gjson_path = Path("../outputs/wa_pcs_cover_all_osm_cities_not_covered_in_b1.geojson")
gjson_path = Path("../datasets/b1-total_area.geojson")
# gjson_path = Path("../datasets/b2_overture_coverage_clipped.geojson")
zip_path = Path("../datasets/EagleView_WA Flight Coverage.zip")

if not gjson_path.exists():
    raise FileNotFoundError(f"GeoJSON not found: {gjson_path}")
if not zip_path.exists():
    raise FileNotFoundError(f"ZIP shapefile not found: {zip_path}")

# Load GeoJSON (#1)
gdf1 = gpd.read_file(str(gjson_path))
if gdf1.empty:
    raise ValueError("GeoJSON layer is empty")

# Load shapefile(s) from the ZIP (#3). Try direct read, otherwise extract.
try:
    gdf3 = gpd.read_file(f"zip://{zip_path}")
except Exception:
    # fallback: extract zip and read the first .shp found
    with tempfile.TemporaryDirectory() as tmpdir:
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(tmpdir)
        shp_files = list(Path(tmpdir).rglob("*.shp"))
        if not shp_files:
            raise FileNotFoundError("No .shp found inside the ZIP")
        gdf3 = gpd.read_file(str(shp_files[0]))

if gdf3.empty:
    raise ValueError("Shapefile layer from ZIP is empty")

# Choose a projected CRS suitable for area calculation.
# Use NAD83 Conus Albers (EPSG:5070) which is appropriate for US-wide datasets.
target_crs = "EPSG:5070"

# If either layer has no CRS, try to set it from the other; if still missing, raise.
if gdf1.crs is None and gdf3.crs is None:
    raise ValueError("Both layers lack CRS information. Cannot proceed.")
if gdf1.crs is None and gdf3.crs is not None:
    gdf1.set_crs(gdf3.crs, inplace=True)
if gdf3.crs is None and gdf1.crs is not None:
    gdf3.set_crs(gdf1.crs, inplace=True)

# Reproject both to target_crs
gdf1_proj = gdf1.to_crs(target_crs)
gdf3_proj = gdf3.to_crs(target_crs)


# Create single (dissolved) geometries for each source to simplify spatial ops
# Clean geometries first (make valid and drop Z), then union with fallbacks to avoid GEOS topology errors.
from shapely.ops import transform as shapely_transform
from shapely.geometry import mapping, shape

def make_valid_2d(geom):
    """Return a valid 2D geometry. Try make_valid, then buffer(0) fallback, and drop Z."""
    if geom is None or geom.is_empty:
        return geom
    try:
        g = make_valid(geom)
    except Exception:
        try:
            g = geom.buffer(0)
        except Exception:
            g = geom
    # drop Z coordinate if present
    def _drop_z(x, y, z=None):
        return (x, y)
    try:
        g2 = shapely_transform(_drop_z, g)
    except Exception:
        g2 = g
    # final attempt to ensure validity
    if not getattr(g2, "is_valid", True):
        try:
            g2 = g2.buffer(0)
        except Exception:
            pass
    return g2

# Apply cleaning to the input GeoDataFrames
gdf1_proj['geometry'] = gdf1_proj.geometry.apply(make_valid_2d)
gdf3_proj['geometry'] = gdf3_proj.geometry.apply(make_valid_2d)

# Drop empty geometries to avoid issues in unary_union
gdf1_proj = gdf1_proj[~gdf1_proj.geometry.is_empty]
gdf3_proj = gdf3_proj[~gdf3_proj.geometry.is_empty]

# Perform unary_union with a fallback that tries buffering each geom if the union fails
try:
    union1 = ops.unary_union(list(gdf1_proj.geometry))
except Exception:
    gdf1_proj['geometry'] = gdf1_proj.geometry.apply(lambda g: g.buffer(0) if g is not None and not g.is_empty else g)
    union1 = ops.unary_union(list(gdf1_proj.geometry))

try:
    union3 = ops.unary_union(list(gdf3_proj.geometry))
except Exception:
    gdf3_proj['geometry'] = gdf3_proj.geometry.apply(lambda g: g.buffer(0) if g is not None and not g.is_empty else g)
    union3 = ops.unary_union(list(gdf3_proj.geometry))

# Ensure the unions themselves are valid 2D geometries
union1 = make_valid_2d(union1)
union3 = make_valid_2d(union3)

# Compute intersection (portion of #1 covered by #3)
covered_geom = union1.intersection(union3) if (union1 is not None and union3 is not None) else None
# Portion of #1 not covered
not_covered_geom = union1.difference(union3) if union1 is not None else None

# Area calculations (units = square meters for EPSG:5070)
total_area_m2 = union1.area if union1 is not None and not union1.is_empty else 0.0
covered_area_m2 = covered_geom.area if covered_geom is not None and not covered_geom.is_empty else 0.0
not_covered_area_m2 = not_covered_geom.area if not_covered_geom is not None and not not_covered_geom.is_empty else 0.0

# Small check: numeric consistency
if abs((covered_area_m2 + not_covered_area_m2) - total_area_m2) > 1e-6:
    # minor floating point tolerance allowed
    pass

# Print summary
print(f"CRS used for area calc: {target_crs}")
print(f"Total area of {gjson_path.name}: {total_area_m2:,.2f} m^2 ({total_area_m2/1e6:,.4f} km^2)")
print(f"Area covered by {zip_path.name}: {covered_area_m2:,.2f} m^2 ({covered_area_m2/1e6:,.4f} km^2)")
print(f"Area NOT covered: {not_covered_area_m2:,.2f} m^2 ({not_covered_area_m2/1e6:,.4f} km^2)")

# Save results as GeoJSONs for inspection
out_dir = Path("../outputs")
covered_gdf = gpd.GeoDataFrame({"id": [1], "area_m2": [covered_area_m2]}, geometry=[covered_geom], crs=target_crs)
not_covered_gdf = gpd.GeoDataFrame({"id": [1], "area_m2": [not_covered_area_m2]}, geometry=[not_covered_geom], crs=target_crs)

covered_fp = out_dir / "covered.geojson"
not_covered_fp = out_dir / "not_covered.geojson"
covered_gdf.to_crs("EPSG:4326").to_file(covered_fp, driver="GeoJSON")      # write in WGS84 for portability
not_covered_gdf.to_crs("EPSG:4326").to_file(not_covered_fp, driver="GeoJSON")

print(f"Wrote covered geometry to: {covered_fp}")
print(f"Wrote not-covered geometry to: {not_covered_fp}")

CRS used for area calc: EPSG:5070
Total area of b1-total_area.geojson: 8,009,499,618.39 m^2 (8,009.4996 km^2)
Area covered by EagleView_WA Flight Coverage.zip: 5,886,661,236.07 m^2 (5,886.6612 km^2)
Area NOT covered: 2,122,838,382.32 m^2 (2,122.8384 km^2)
Wrote covered geometry to: ../outputs/covered.geojson
Wrote not-covered geometry to: ../outputs/not_covered.geojson


In [17]:
vexcel_tiles_needed = not_covered_area_m2 / (256 * 256 * 0.15*0.15)
print(f"Vexcel tiles needed: {vexcel_tiles_needed:.0f}")

Vexcel tiles needed: 1439642
